In [9]:
import numpy as np
U64 = np.uint64

In [83]:
def bit2int(bb):
    return np.sum(bb * (1 << np.arange(64, dtype=U64).reshape(8,8)[::-1]))

def int2bit(bb):
    return (((1 << np.arange(64, dtype=U64)) & bb) > 0).astype(U64).reshape(8, 8)[::-1]

def square(index):
    return np.ones(1, dtype=U64)[0] << index

zero = square(40) * 0
one = square(0)

## PCG64

In [166]:
class PCG64:
    def __init__(self, seed=np.uint64(0x853c49e6748fea9b)):
        self.state = np.uint64(0)
        self.multiplier = np.uint64(6364136223846793005)
        self.increment = np.uint64(1442695040888963407)
        self.seed_rng(seed)

    def seed_rng(self, seed):
        self.state = np.uint64(0)
        self.next()
        self.state += seed
        self.next()

    def next(self):
        oldstate = self.state
        with np.errstate(over='ignore'):
            self.state = self.state * self.multiplier + self.increment

        xorshifted = ((oldstate >> 18) ^ oldstate) >> 27
        rot = oldstate >> 59
        with np.errstate(over='ignore'):
            res = (xorshifted >> rot) | (xorshifted << ((-rot) & 63))
        return res

    def __call__(self):
        return self.next()

In [167]:
rng = PCG64();

In [181]:
oldstate = rng.state
xorshifted = ((oldstate >> 18) ^ oldstate) >> 27
bin(xorshifted)

'0b110110011011100110111101000010110111'

In [236]:
bin(6917810555231207424)[2:].zfill(64)

'0110000000000001000000000000110001000000000000010000000000000000'

In [194]:
len("0000000000000000000000000000")

28

## Pawn attack

In [67]:
bb = np.ones((8, 8), dtype = np.uint64)
bb[:, 0] = 0
not_a_file = bit2int(bb)
print("not_a_file =", bit2int(bb))

not_a_file = 18374403900871474942


In [68]:
bb = np.ones((8, 8), dtype = np.uint64)
bb[:, 7] = 0
not_h_file = bit2int(bb)
print("not_h_file =", bit2int(bb))

not_h_file = 9187201950435737471


In [69]:
bb = np.ones((8, 8), dtype = np.uint64)
bb[:, [6,7]] = 0
not_gh_file = bit2int(bb)
print("not_gh_file =", bit2int(bb))

not_gh_file = 4557430888798830399


In [70]:
bb = np.ones((8, 8), dtype = np.uint64)
bb[:, [0,1]] = 0
not_ab_file = bit2int(bb)
print("not_ab_file =", bit2int(bb))

not_ab_file = 18229723555195321596


In [122]:
pawn_attacks = np.zeros((2, 64), dtype=U64);
# black
for j in range(64):
    attacks = zero.copy();
    bb = square(j);
    if ((bb >> 7)& not_a_file): attacks |= (bb >> 7);
    if ((bb >> 9)& not_h_file): attacks |= (bb >> 9);
    pawn_attacks[0, j] = attacks

# white
for j in range(64):
    attacks = zero.copy();
    bb = square(j);
    if ((bb << 7)& not_h_file): attacks |= (bb << 7);
    if ((bb << 9)& not_a_file): attacks |= (bb << 9);
    pawn_attacks[1, j] = attacks

res = "{{"
for j in range(64):
    res += str(pawn_attacks[0, j]) + "ULL"
    if j < 63: res += ","
res += "},{"
for j in range(64):
    res += str(pawn_attacks[1, j]) + "ULL"
    if j < 63: res += ","
res += "}}"
res

'{{0ULL,0ULL,0ULL,0ULL,0ULL,0ULL,0ULL,0ULL,2ULL,5ULL,10ULL,20ULL,40ULL,80ULL,160ULL,64ULL,512ULL,1280ULL,2560ULL,5120ULL,10240ULL,20480ULL,40960ULL,16384ULL,131072ULL,327680ULL,655360ULL,1310720ULL,2621440ULL,5242880ULL,10485760ULL,4194304ULL,33554432ULL,83886080ULL,167772160ULL,335544320ULL,671088640ULL,1342177280ULL,2684354560ULL,1073741824ULL,8589934592ULL,21474836480ULL,42949672960ULL,85899345920ULL,171798691840ULL,343597383680ULL,687194767360ULL,274877906944ULL,2199023255552ULL,5497558138880ULL,10995116277760ULL,21990232555520ULL,43980465111040ULL,87960930222080ULL,175921860444160ULL,70368744177664ULL,562949953421312ULL,1407374883553280ULL,2814749767106560ULL,5629499534213120ULL,11258999068426240ULL,22517998136852480ULL,45035996273704960ULL,18014398509481984ULL},{512ULL,1280ULL,2560ULL,5120ULL,10240ULL,20480ULL,40960ULL,16384ULL,131072ULL,327680ULL,655360ULL,1310720ULL,2621440ULL,5242880ULL,10485760ULL,4194304ULL,33554432ULL,83886080ULL,167772160ULL,335544320ULL,671088640ULL,13421

In [125]:
int2bit(pawn_attacks[0, 46])

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]], dtype=uint64)

## Knights atacks

In [128]:
knight_attacks = np.zeros((64), dtype=U64);
# white
for j in range(64):
    attacks = zero.copy();
    bb = square(j);
    if ((bb >> 17) & not_h_file):  attacks |= (bb >> 17);
    if ((bb >> 15) & not_a_file):  attacks |= (bb >> 15);
    if ((bb >> 10) & not_gh_file): attacks |= (bb >> 10);
    if ((bb >> 6)  & not_ab_file): attacks |= (bb >> 6);
    if ((bb << 17) & not_a_file):  attacks |= (bb << 17);
    if ((bb << 15) & not_h_file):  attacks |= (bb << 15);
    if ((bb << 10) & not_ab_file): attacks |= (bb << 10);
    if ((bb << 6)  & not_gh_file): attacks |= (bb << 6);
    knight_attacks[j] = attacks

res = "{"
for j in range(64):
    res += str(knight_attacks[j]) + "ULL"
    if j < 63: res += ","
res += "}"
res

'{132096ULL,329728ULL,659712ULL,1319424ULL,2638848ULL,5277696ULL,10489856ULL,4202496ULL,33816580ULL,84410376ULL,168886289ULL,337772578ULL,675545156ULL,1351090312ULL,2685403152ULL,1075839008ULL,8657044482ULL,21609056261ULL,43234889994ULL,86469779988ULL,172939559976ULL,345879119952ULL,687463207072ULL,275414786112ULL,2216203387392ULL,5531918402816ULL,11068131838464ULL,22136263676928ULL,44272527353856ULL,88545054707712ULL,175990581010432ULL,70506185244672ULL,567348067172352ULL,1416171111120896ULL,2833441750646784ULL,5666883501293568ULL,11333767002587136ULL,22667534005174272ULL,45053588738670592ULL,18049583422636032ULL,145241105196122112ULL,362539804446949376ULL,725361088165576704ULL,1450722176331153408ULL,2901444352662306816ULL,5802888705324613632ULL,11533718717099671552ULL,4620693356194824192ULL,288234782788157440ULL,576469569871282176ULL,1224997833292120064ULL,2449995666584240128ULL,4899991333168480256ULL,9799982666336960512ULL,1152939783987658752ULL,2305878468463689728ULL,11280989300981

## King attacks

In [133]:
king_attacks = np.zeros((64), dtype=U64);
# white
for j in range(64):
    attacks = zero.copy();
    bb = square(j);
    if (bb >> 8): attacks |= (bb >> 8);
    if ((bb >> 9) & not_h_file): attacks |= (bb >> 9);
    if ((bb >> 7) & not_a_file): attacks |= (bb >> 7);
    if ((bb >> 1) & not_h_file): attacks |= (bb >> 1);
    if (bb << 8): attacks |= (bb << 8);
    if ((bb << 9) & not_a_file): attacks |= (bb << 9);
    if ((bb << 7) & not_h_file): attacks |= (bb << 7);
    if ((bb << 1) & not_a_file): attacks |= (bb << 1);
    king_attacks[j] = attacks

res = "{"
for j in range(64):
    res += str(king_attacks[j]) + "ULL"
    if j < 63: res += ","
res += "}"
res

'{770ULL,1797ULL,3594ULL,7188ULL,14376ULL,28752ULL,57504ULL,49216ULL,197123ULL,460039ULL,920078ULL,1840156ULL,3680312ULL,7360624ULL,14721248ULL,12599488ULL,50463488ULL,117769984ULL,235539968ULL,471079936ULL,942159872ULL,1884319744ULL,3768639488ULL,3225468928ULL,12918652928ULL,30149115904ULL,60298231808ULL,120596463616ULL,241192927232ULL,482385854464ULL,964771708928ULL,825720045568ULL,3307175149568ULL,7718173671424ULL,15436347342848ULL,30872694685696ULL,61745389371392ULL,123490778742784ULL,246981557485568ULL,211384331665408ULL,846636838289408ULL,1975852459884544ULL,3951704919769088ULL,7903409839538176ULL,15806819679076352ULL,31613639358152704ULL,63227278716305408ULL,54114388906344448ULL,216739030602088448ULL,505818229730443264ULL,1011636459460886528ULL,2023272918921773056ULL,4046545837843546112ULL,8093091675687092224ULL,16186183351374184448ULL,13853283560024178688ULL,144959613005987840ULL,362258295026614272ULL,724516590053228544ULL,1449033180106457088ULL,2898066360212914176ULL,579613272

## Bishop (mask)

In [141]:
def mask_bishop_attacks(square: int) -> np.uint64:
    # result attacks bitboard
    attacks = np.uint64(0)
    
    # init target rank & files
    tr = square // 8
    tf = square % 8
    
    # mask relevant bishop occupancy bits
    for r, f in zip(range(tr + 1, 7), range(tf + 1, 7)):
        attacks |= np.uint64(1) << np.uint64(r * 8 + f)
    for r, f in zip(range(tr - 1, 0, -1), range(tf + 1, 7)):
        attacks |= np.uint64(1) << np.uint64(r * 8 + f)
    for r, f in zip(range(tr + 1, 7), range(tf - 1, 0, -1)):
        attacks |= np.uint64(1) << np.uint64(r * 8 + f)
    for r, f in zip(range(tr - 1, 0, -1), range(tf - 1, 0, -1)):
        attacks |= np.uint64(1) << np.uint64(r * 8 + f)
    
    # return attack map
    return attacks

In [142]:
bishop_masks = np.zeros((64), dtype=U64);
# white
for j in range(64):
    attacks = zero.copy();
    bishop_masks[j] = mask_bishop_attacks(j)

res = "{"
for j in range(64):
    res += str(bishop_masks[j]) + "ULL"
    if j < 63: res += ","
res += "}"
res

'{18049651735527936ULL,70506452091904ULL,275415828992ULL,1075975168ULL,38021120ULL,8657588224ULL,2216338399232ULL,567382630219776ULL,9024825867763712ULL,18049651735527424ULL,70506452221952ULL,275449643008ULL,9733406720ULL,2216342585344ULL,567382630203392ULL,1134765260406784ULL,4512412933816832ULL,9024825867633664ULL,18049651768822272ULL,70515108615168ULL,2491752130560ULL,567383701868544ULL,1134765256220672ULL,2269530512441344ULL,2256206450263040ULL,4512412900526080ULL,9024834391117824ULL,18051867805491712ULL,637888545440768ULL,1135039602493440ULL,2269529440784384ULL,4539058881568768ULL,1128098963916800ULL,2256197927833600ULL,4514594912477184ULL,9592139778506752ULL,19184279556981248ULL,2339762086609920ULL,4538784537380864ULL,9077569074761728ULL,562958610993152ULL,1125917221986304ULL,2814792987328512ULL,5629586008178688ULL,11259172008099840ULL,22518341868716544ULL,9007336962655232ULL,18014673925310464ULL,2216338399232ULL,4432676798464ULL,11064376819712ULL,22137335185408ULL,44272556441600

In [248]:
bishop_relevant_bits = np.bitwise_count(bishop_masks)
bishop_relevant_bits.reshape(8,8)

array([[6, 5, 5, 5, 5, 5, 5, 6],
       [5, 5, 5, 5, 5, 5, 5, 5],
       [5, 5, 7, 7, 7, 7, 5, 5],
       [5, 5, 7, 9, 9, 7, 5, 5],
       [5, 5, 7, 9, 9, 7, 5, 5],
       [5, 5, 7, 7, 7, 7, 5, 5],
       [5, 5, 5, 5, 5, 5, 5, 5],
       [6, 5, 5, 5, 5, 5, 5, 6]], dtype=uint8)

## Rook (mask)

In [144]:
def mask_rook_attacks(square: int) -> np.uint64:
    # result attacks bitboard
    attacks = np.uint64(0)
    
    # init target rank & file
    tr = square // 8
    tf = square % 8
    
    # mask relevant rook occupancy bits
    for r in range(tr + 1, 7):
        attacks |= np.uint64(1) << np.uint64(r * 8 + tf)
    for r in range(tr - 1, 0, -1):
        attacks |= np.uint64(1) << np.uint64(r * 8 + tf)
    for f in range(tf + 1, 7):
        attacks |= np.uint64(1) << np.uint64(tr * 8 + f)
    for f in range(tf - 1, 0, -1):
        attacks |= np.uint64(1) << np.uint64(tr * 8 + f)
    
    # return attack map
    return attacks

In [145]:
rook_masks = np.zeros((64), dtype=U64);
# white
for j in range(64):
    attacks = zero.copy();
    rook_masks[j] = mask_rook_attacks(j)

res = "{"
for j in range(64):
    res += str(rook_masks[j]) + "ULL"
    if j < 63: res += ","
res += "}"
res

'{282578800148862ULL,565157600297596ULL,1130315200595066ULL,2260630401190006ULL,4521260802379886ULL,9042521604759646ULL,18085043209519166ULL,36170086419038334ULL,282578800180736ULL,565157600328704ULL,1130315200625152ULL,2260630401218048ULL,4521260802403840ULL,9042521604775424ULL,18085043209518592ULL,36170086419037696ULL,282578808340736ULL,565157608292864ULL,1130315208328192ULL,2260630408398848ULL,4521260808540160ULL,9042521608822784ULL,18085043209388032ULL,36170086418907136ULL,282580897300736ULL,565159647117824ULL,1130317180306432ULL,2260632246683648ULL,4521262379438080ULL,9042522644946944ULL,18085043175964672ULL,36170086385483776ULL,283115671060736ULL,565681586307584ULL,1130822006735872ULL,2261102847592448ULL,4521664529305600ULL,9042787892731904ULL,18085034619584512ULL,36170077829103616ULL,420017753620736ULL,699298018886144ULL,1260057572672512ULL,2381576680245248ULL,4624614895390720ULL,9110691325681664ULL,18082844186263552ULL,36167887395782656ULL,35466950888980736ULL,34905104758997504

In [148]:
len(bin(4095)[2:])

12

In [249]:
rook_relevant_bits = np.bitwise_count(rook_masks)
rook_relevant_bits.reshape(8,8)

array([[12, 11, 11, 11, 11, 11, 11, 12],
       [11, 10, 10, 10, 10, 10, 10, 11],
       [11, 10, 10, 10, 10, 10, 10, 11],
       [11, 10, 10, 10, 10, 10, 10, 11],
       [11, 10, 10, 10, 10, 10, 10, 11],
       [11, 10, 10, 10, 10, 10, 10, 11],
       [11, 10, 10, 10, 10, 10, 10, 11],
       [12, 11, 11, 11, 11, 11, 11, 12]], dtype=uint8)

## Generate magic numbers

In [245]:
rng = np.random.Generator(np.random.PCG64DXSM())

In [246]:
rnd = rng.integers(low=0, high=np.iinfo(np.uint64).max, size=(1000, 1000), dtype=np.uint64)

In [ ]:
0100000 0 0 0 1 0 0 

In [251]:
0b0100000000100

2052

In [252]:
64 * 4095 * 8 * 1e-6

2.09664

In [66]:
bb = np.ones((8, 8), dtype = np.uint64)
bb[0] = 0
# print(bit2int(bb))
(int2bit(bit2int(bb)) == bb).all()

np.True_

In [82]:

one

np.uint64(1)

In [43]:
pawn_attacks = np.zeros((2, 64), dtype=U64)

In [40]:
bb = np.zeros((8, 8), dtype = np.uint64)
bb[2, 5] = 1
print(bit2int(bb))

35184372088832


In [30]:
import torch
import time

In [44]:
N = 256
R = 100

for device in (torch.device("cpu"),):
    for dtype in (torch.float64, torch.float32, torch.float16):
        ts = torch.zeros((R,), dtype=torch.float64)
        for j in range(R):
            M = torch.rand((N,N), dtype=dtype,device=device) - 1/2
            v1 = torch.rand((1,N), dtype=dtype,device=device) - 1/2

            t1 = time.perf_counter()
            v2 = torch.matmul(v1, M)
            t2 = time.perf_counter()

            ts[j] = (t2-t1) * 1e3

        print(f"{str(dtype):15}|  {str(device):6}|  {int(round(1e3*ts.mean().item()))} μs")

torch.float64  |  cpu   |  29 μs
torch.float32  |  cpu   |  16 μs
torch.float16  |  cpu   |  121 μs
